# Decision Tree Classification

## Importing the libraries

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score

## Importing the dataset

In [2]:
# Importing dataset
dataset = pd.read_csv(r"C:\Users\55149\Documents\Part 3 - Classification\data_all.csv", index_col=0)

# Creating column for detecting rain events (1) and no rain (0)
dataset['rain'] = 0

# Rain events only occur if total rain is greater than 0.2 mm
mask = dataset.total_rain > 0.2
rain = dataset.loc[mask].index
dataset.loc[rain, 'rain'] = 1

# Selecting columns of interest
dataset = dataset[['rain_class', 'rain'] + [str(i) for i in range(51)]]

# Set the random seed for reproducibility
np.random.seed(42)

# Filter the dataset to select rows where 'rain' column is equal to zero
zero_rain_rows = dataset[dataset['rain'] == 0]

# Randomly select 766 rows from the zero_rain_rows
selected_rows = zero_rain_rows.sample(n=766)

# Combine the selected rows with other rows where 'rain' column is equal to one
final_dataset = pd.concat([selected_rows, dataset[dataset['rain'] == 1]])

# Print the value counts of 'rain_class' in the final dataset
print(final_dataset['rain_class'].value_counts())

dataset = final_dataset

no rain     766
moderate    317
light       301
heavy       142
violent      10
Name: rain_class, dtype: int64


## Hyperparameter tuning with GridSearchCV

In [53]:
# Selecting X and y variables
# X corresponds to power spectrum density values
# y corresponds to total rain
X = dataset[dataset.columns[2:]].iloc[:, :].values
y = dataset.rain.values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [20, 50, 100, 300],
    'learning_rate': [0,75, 0.25, 0.1, 0.01],
    'max_depth': [3, 5, 7, 9],
    'max_features': [3, 5, 7, 10]
}

# Create the gradient boosting classifier
gb_clf = GradientBoostingClassifier()

# Create GridSearchCV object
grid_search = GridSearchCV(gb_clf, param_grid, cv=5, scoring='accuracy')

# Fit the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Best Parameters:  {'learning_rate': 0.25, 'max_depth': 7, 'max_features': 3, 'n_estimators': 20}
Best Score:  0.9748277809147374


## T

In [57]:
# Define a list to store accuracy values
accuracy_values = []
false_positive_values = []
false_negative_values = []
precision_values = []

# Define the number of iterations
num_iterations = 100

for _ in range(num_iterations):
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
    
    # Applying Gradient Boosting Classifier
    classifier = GradientBoostingClassifier(learning_rate=0.25, max_depth=7, max_features=3, n_estimators=20)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    # Get the list of class labels
    class_labels = dataset['rain_class'].unique()
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    false_positive = cm[0][1]  # False positives are in the first row, second column
    false_negative = cm[1][0]  # False negatives are in the second row, first column
    
    precision = precision_score(y_test, y_pred)  # Calculate precision
    
    accuracy_values.append(accuracy)
    false_positive_values.append(false_positive)
    false_negative_values.append(false_negative)
    precision_values.append(precision)

# Calculate the average accuracy
average_accuracy = np.mean(accuracy_values)

# Calculate the average precision
average_precision = np.mean(precision_values)

# Calculate the average false positives and false negatives
average_false_positive = np.mean(false_positive_values)
average_false_negative = np.mean(false_negative_values)

# Print the average accuracy, false positives, false negatives, and precision
print("Average Accuracy:", average_accuracy)
print("Average Precision:", average_precision)
print("Average False Positives:", average_false_positive)
print("Average False Negatives:", average_false_negative)

Average Accuracy: 0.9679947916666667
Average Precision: 0.9580392300425308
Average False Positives: 8.29
Average False Negatives: 4.0


In [60]:
# After predicting the values using the trained classifier, create a new DataFrame
df_results = pd.DataFrame(X_test, columns=dataset.columns[2:])
df_results['predicted'] = y_pred
df_results['actual'] = y_test

# Get the indices of misclassified rows
misclassified_df = df_results[df_results['predicted'] != df_results['actual']]

In [61]:
misclassified_df

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,predicted,actual
1,26.803503,35.327470,1525.432700,5899.313500,2234.192400,184.725450,81.175360,115.506920,228.424880,264.089870,...,170.604630,137.611330,94.244230,72.644260,62.046220,50.863914,44.866432,38.228294,1,0
31,4.137597,3.838700,48.788220,372.848000,332.704200,33.392017,6.906498,10.658554,50.052870,74.567530,...,12.929351,11.506581,10.448172,9.834924,8.857118,7.740404,6.730504,6.048560,0,1
58,4.097923,6.857848,194.160355,826.210815,344.221527,31.922941,13.449718,22.968195,48.732937,77.582993,...,24.291454,23.280214,23.481419,23.368423,21.861965,18.674250,16.542467,15.177099,0,1
139,20.892109,19.467464,489.268370,2760.036400,1528.888900,110.879470,66.372680,158.353740,291.604680,294.301640,...,26.927492,22.884241,19.942509,16.439856,14.472115,12.850518,11.862374,11.204401,0,1
151,37.781540,23.449396,454.029240,3542.621600,2723.361600,235.777270,52.631847,81.710990,245.346590,487.820070,...,51.519010,43.705530,34.929370,30.711810,28.382023,24.757416,21.944790,19.580286,1,0
203,66.781685,55.289986,1781.905396,10387.281250,5497.972656,338.695404,120.944382,180.021057,194.078644,269.786926,...,17.027248,31.904074,52.096306,66.782982,82.563599,102.435799,111.506859,103.013588,1,0
237,119.229780,166.803270,3286.824200,7332.133000,8201.739000,1944.774900,226.636050,159.185460,444.043330,1217.030800,...,119.323060,179.606300,179.153370,168.150470,161.035320,144.573460,131.775790,141.618960,1,0
249,64.338554,83.336260,2249.990500,9949.998000,4365.508300,332.242680,131.780730,193.267330,468.586150,961.345760,...,61.802944,61.367565,44.413944,35.665490,31.328625,28.474276,27.724882,26.920805,1,0
274,11.117081,9.675589,338.833200,1864.327000,943.687500,271.359040,127.385710,401.830720,674.184450,309.454000,...,30.072775,51.357710,59.952020,58.175816,60.777360,62.773376,61.302666,54.113377,1,0
339,35.406567,32.099865,956.121800,5322.316000,2719.755900,438.229680,127.912380,65.984040,117.808720,206.658650,...,155.321430,160.399140,164.964430,169.737300,148.876720,109.931740,85.984910,73.875750,1,0
